In [14]:
import os
import re
import yaml
import torch
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display


from dataset import JetSubstructureDataset
from models import JetSubstructureNeqModel
from ensemble import AveragingJetNeqModel, BaggingJetNeqModel, AdaBoostJetNeqModel

In [15]:
dataset_file = "data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_truth.z"
dataset_config = "config/yaml_IP_OP_config.yml"

In [23]:
def set_random_seeds(exp_config, cuda=False):
    with open(exp_config, "r") as f:
        config = yaml.safe_load(f)
    # Set random seeds
    random.seed(config["seed"])
    np.random.seed(config["seed"])
    torch.manual_seed(config["seed"])
    os.environ["PYTHONHASHSEED"] = str(config["seed"])
    if cuda:
        torch.cuda.manual_seed_all(config["seed"])
        torch.backends.cudnn.deterministic = True

def load_dataset(dataset_file, dataset_config):
     # Fetch the datasets
    dataset = {}
    dataset["train"] = JetSubstructureDataset(
        dataset_file, dataset_config, split="train"
    )
    # This dataset is so small, we'll just use the training set as the validation set, otherwise we may have too few trainings examples to converge.
    dataset["valid"] = JetSubstructureDataset(
        dataset_file, dataset_config, split="train"
    )
    dataset["test"] = JetSubstructureDataset(
        dataset_file, dataset_config, split="test"
    )
    return dataset

def load_model(exp_config, dataset, checkpoint=None):
    with open(exp_config, "r") as f:
        config = yaml.safe_load(f)
    # Instantiate model
    x, y = dataset["train"][0]
    config["input_length"] = len(x)
    config["output_length"] = len(y)
    print(f"Input length: {config['input_length']}")
    print(f"Output length: {config['output_length']}")

    # Ensemble settings
    quantize_avg = False
    if "quantize_avg" in config:
        quantize_avg = config["quantize_avg"]
    if "post_transform_output" not in config:
        config["post_transform_output"] = True # Default
    if "same_output_scale" not in config:
        config["same_output_scale"] = False # Default
    
    if "ensemble_method" in config:
        if config["ensemble_method"] == "averaging":
            print("Averaging ensemble method")
            model = AveragingJetNeqModel(
                config, config["ensemble_size"], quantize_avg=quantize_avg
            )
        elif config["ensemble_method"] == "bagging":
            print("Bagging ensemble method")
            if "independent" not in config:
                config["independent"] = False # Default
            model = BaggingJetNeqModel(
                config,
                config["ensemble_size"],
                quantize_avg=quantize_avg,
                single_model_mode=False,
            )
        elif config["ensemble_method"] == "adaboost":
            print("AdaBoost ensemble method")
            if "independent" not in config:
                config["independent"] = False # Default
            model = AdaBoostJetNeqModel(
                config,
                config["ensemble_size"],
                len(dataset["train"]),
                quantize_avg=quantize_avg,
                single_model_mode=False,
            )
        else:
            raise ValueError(f"Unknown ensemble method: {config['ensemble_method']}")
    else:  # Single model learning
        model = JetSubstructureNeqModel(config)
    if checkpoint is not None:
        print(f"Loading pre-trained checkpoint {checkpoint}")
        checkpoint = torch.load(checkpoint, map_location="cpu")
        model.load_state_dict(checkpoint["model_dict"])

    print(f"Model: {model.__class__.__name__}")

    return model

## Averaging Sparsity Masks

In [24]:
config = "./averaging/averaging_small_ensemble_size2/hparams.yml"
ckpt="./averaging/averaging_small_ensemble_size2/best_accuracy.pth"
set_random_seeds(config)
dataset = load_dataset(dataset_file, dataset_config)
model = load_model(config, dataset, checkpoint=ckpt)

Input length: 16
Output length: 5
Averaging ensemble method
Loading pre-trained checkpoint ./averaging/averaging_small_ensemble_size2/best_accuracy.pth
Model: AveragingJetNeqModel


In [28]:
# TODO: Compute Cartesian distance between input sparsity masks of the ensemble members
def compute_cartesian_distance(model):
    ensemble_size = model.num_models
    input_masks = []
    for i in range(ensemble_size):
        mask = model.ensemble[i].module_list[0].fc.mask.mask.numpy()
        input_masks.append(mask)
    input_masks = np.array(input_masks)
    print(input_masks.shape)
    print(input_masks)
    cartesian_distances = np.zeros((ensemble_size, ensemble_size))
    for i in range(ensemble_size):
        for j in range(ensemble_size):
            cartesian_distances[i, j] = np.linalg.norm(input_masks[i] - input_masks[j])
    total_distance = np.sum(cartesian_distances)
    return cartesian_distances, total_distance
dists, total_dist = compute_cartesian_distance(model)

(2, 64, 16)
[[[0. 1. 0. ... 1. 0. 0.]
  [0. 1. 0. ... 0. 0. 1.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 1. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [29]:
dists, total_dist

(array([[ 0.        , 17.43559647],
        [17.43559647,  0.        ]]),
 34.871192932128906)

In [ ]:
# TODO: Load all averaging models and compute the total Cartesian distance between their input sparsity masks
def compute_total_cartesian_distance(exp_dir):
    config = f"{exp_dir}/hparams.yml"
    ckpt = f"{exp_dir}/best_accuracy.pth"
    set_random_seeds(config)
    dataset = load_dataset(dataset_file, dataset_config)
    model = load_model(config, dataset, checkpoint=ckpt)
    dists, total_dist = compute_cartesian_distance(model)
    return total_dist

# Compute for every model in the directory
